In [1]:
import pandas as pd

In [2]:
print("me")

me


In [3]:
df = pd.read_csv("adjust_sample_data.csv").sort_values(by='created_at')

In [4]:
df.info()
df.head(5)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 770529 entries, 215200 to 320513
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   adid           770529 non-null  object
 1   network_name   770529 non-null  object
 2   installed_at   770529 non-null  object
 3   activity_kind  770529 non-null  object
 4   event_name     681147 non-null  object
 5   created_at     770529 non-null  object
dtypes: object(6)
memory usage: 41.2+ MB


,adid,network_name,installed_at,activity_kind,event_name,created_at
215200,db52b12c9c35e053bdb071b434c8dac4,Cafebazaar-searchad,7/9/2019 23:48,event,get_contact,7/10/2019 0:00
101844,f4f4e63a4c6913ae714bdaeaab6d7002,Organic,7/6/2019 1:59,event,get_contact,7/10/2019 0:00
606061,2a4de36964a24f8f9261a2c8de806ee9,Cafebazaar-searchad,7/7/2019 21:44,event,get_contact,7/10/2019 0:00
101692,1727d32e0a7bf27458275bfe58110b2f,Organic,7/9/2019 18:09,event,get_contact,7/10/2019 0:01
606726,3dbfd4dfda300b1c11cf2695bf38ebf8,Cafebazaar-searchad,7/8/2019 8:07,event,new_post_submit,7/10/2019 0:01


In [5]:
df['installed_at'] = pd.to_datetime(df['installed_at'])
df['created_at'] = pd.to_datetime(df['created_at'])

In [6]:
df['aqz_age'] = df['created_at'] - df['installed_at']
df['aqz_age_day'] = df['aqz_age'].dt.days
df['aqz_cohort_day'] = df['installed_at'].dt.date

In [7]:
df_users = df.drop_duplicates('adid', keep='first')
df_users = df_users[['adid','network_name','installed_at']]
df_users.set_index('adid',drop=True,inplace=True)

In [8]:
df=df[df['activity_kind']=='event']
df_conversion = df.drop_duplicates('adid',keep='first')
df_conversion.set_index('adid',drop=True,inplace=True)
df.set_index('adid',drop=True,inplace=True)

In [9]:
df['cnv_date'] = df_conversion['created_at']
df['cnv_cohort_day'] = df['cnv_date'].dt.date
df['cnv_age'] = df['created_at'] - df['cnv_date']
df['cnv_age_day'] = df['cnv_age'].dt.days

# Data Cleaning

# Exploration